In [ ]:
import pandas as pd
import numpy as np


df= pd.read_csv('credit_card_transactions.csv')
# Display the first few rows of the dataframe
print(df.head())


In [ ]:
# tạo các bảng dim_time, dim_customer, dim_merchant, dim_category, fact_transactions, fact_merchant_time
from datetime import datetime

df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])

df_time = (
    df[["trans_date_trans_time", "unix_time"]].drop_duplicates().reset_index(drop=True)
)

# Epoch at start of day
df_time["epoch_day"] = (df_time["unix_time"] // 86400) * 86400

# Date parts
df_time["date"] = df_time["trans_date_trans_time"].dt.date
df_time["day"] = df_time["trans_date_trans_time"].dt.day
df_time["month"] = df_time["trans_date_trans_time"].dt.month
df_time["day_of_week"] = df_time["trans_date_trans_time"].dt.dayofweek
df_time["quarter"] = df_time["trans_date_trans_time"].dt.quarter
df_time["year"] = df_time["trans_date_trans_time"].dt.year
df_time["monthKey"] = df_time["year"].astype(str).str.zfill(4) + df_time[
    "month"
].astype(str).str.zfill(2)

df_time.drop(columns=["trans_date_trans_time", "unix_time"], inplace=True)
df_time.drop_duplicates(inplace=True)
df_time.reset_index(drop=True, inplace=True)
df_time.head(10)

In [ ]:
# tao bang dim_customer
df_customer = (
    df[["cc_num", "first", "last", "gender", "city", "job", "dob"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
df_customer["full_name"] = df_customer["first"] + " " + df_customer["last"]
df_customer.head(6)

In [ ]:
# tao bang dim category
df_category = df[["category"]].drop_duplicates().reset_index(drop=True)
df_category["id"] = "cat" + df_category.index.astype(str).str.zfill(3)
df_category.head(6)

In [ ]:
# tao bang dim_merchant
df_merchant = df[["merchant"]].drop_duplicates().reset_index(drop=True)
df_merchant["merchant"] = (
    df_merchant["merchant"].str.split(r"[, ]").str.join(" ").str.lower().str.strip()
)
df_merchant["merchant_id"] = "mer" + df_merchant.index.astype(str).str.zfill(4)

df_merchant.count()

In [ ]:
# tao bảng fact_transactions
df["epoch_day"] = (df["unix_time"] // 86400) * 86400
df["merchant"] = df["merchant"].str.split(r"[, ]").str.join(" ").str.lower().str.strip()

merchant_mapping = df_merchant.set_index("merchant")["merchant_id"]
df["merchant_id"] = df["merchant"].map(merchant_mapping)
df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])
df["category"] = df["category"].str.lower().str.strip()

category_mapping = df_category.set_index("category")["id"]
df["category_id"] = df["category"].map(category_mapping)
df["dob"] = pd.to_datetime(df["dob"])
df["cus_age"] = df["dob"].apply(lambda x: datetime.now().year - x.year)
bins = [0, 18, 26, 36, 46, 56, 66, 200]
labels_name = ["Under 18", "18-25", "26-35", "36-45", "46-55", "56-65", "66 and above"]

# Tạo cột age_group_name
df["age_group_name"] = pd.cut(df["cus_age"], bins=bins, labels=labels_name, right=False)
# Tạo age_group_id theo thứ tự nhóm, dạng age00, age01,..
df["age_group_id"] = df["age_group_name"].apply(
    lambda x: f"age{labels_name.index(x)+0:02d}" if pd.notna(x) else "age_unknown"
)
# tao dim_age_group
dim_age_group = df[["age_group_name"]].drop_duplicates().reset_index(drop=True)
dim_age_group["age_group_id"] = df["age_group_id"].unique()

df_new = df.drop(
    columns=[
        "first",
        "last",
        "gender",
        "city",
        "job",
        "dob",
        "merchant",
        "state",
        "zip",
        "lat",
        "long",
        "unix_time",
        "merch_lat",
    ],
    inplace=False,
)
df_fact = df_new[
    [
        "trans_num",
        "cc_num",
        "trans_date_trans_time",
        "amt",
        "category_id",
        "merchant_id",
        "epoch_day",
        "age_group_id",
    ]
]
df_fact.head(6)

In [8]:
df_fact_merchant = (
    df_fact.groupby(["merchant_id", "epoch_day"])
    .agg(
        total_amount=pd.NamedAgg(column="amt", aggfunc="sum"),
        transaction_count=pd.NamedAgg(column="trans_num", aggfunc="count"),
    )
    .reset_index()
)


In [ ]:
import psycopg2
from psycopg2.extras import execute_values

# Hàm helper để insert dữ liệu
def insert_data(table_name, df):
    conn = psycopg2.connect(host='localhost', database='mydatabase', user='admin', password='admin123', port='5432')
    cur = conn.cursor()
    cols = ','.join(list(df.columns))
    values = list(df.itertuples(index=False, name=None))
    sql = f"INSERT INTO {table_name} ({cols}) VALUES %s"
    try:
        execute_values(cur, sql, values, page_size=1000)
        conn.commit()
        print(f"✓ Data for {table_name} inserted successfully ({len(values)} rows)")
    except Exception as e:
        conn.rollback()
        print(f"✗ Error inserting data for {table_name}: {e}")
    finally:
        cur.close()
        conn.close()

# Insert tất cả các bảng
insert_data('fact_merchant', df_fact_merchant)
insert_data('dim_time', df_time)
insert_data('dim_customer', df_customer)
insert_data('dim_merchant', df_merchant)
insert_data('dim_category', df_category)
insert_data('dim_age_group', dim_age_group)

# Insert fact_transactions (loại bỏ cột trans_date_trans_time trước)
df_fact_copy = df_fact.drop(columns=['trans_date_trans_time'])
insert_data('fact_transactions', df_fact_copy)

    

    